In [1]:
from notebook_inits import *
import polars as pl

In [2]:
from main import fetch_all_data

In [3]:
data = fetch_all_data()

fetching exchange data.. https://api.exchange.coinbase.com/p
cb fetch complete
fetching exchange data.. https://api.gemini.com/v1/book/BTCU
Finished processing and saving data for: cb
gem fetch complete
Finished processing and saving data for: gem


In [4]:
cb_bids = pl.DataFrame(data['cb']['bids'], schema=['price', 'qty', 'orders'], orient="row")
cb_asks = pl.DataFrame(data['cb']['asks'], schema=['price', 'qty', 'orders'], orient="row")
_, _ = cb_bids.drop_in_place('orders'), cb_asks.drop_in_place('orders')
cb_bids, cb_asks = cb_bids.cast(pl.Float64), cb_asks.cast(pl.Float64)

In [5]:
cb_bids.head(3), cb_asks.head(3)

(shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 103744.42 ┆ 0.078087 │
 │ 103743.48 ┆ 0.002883 │
 │ 103743.36 ┆ 0.000011 │
 └───────────┴──────────┘,
 shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 103744.43 ┆ 0.067058 │
 │ 103748.0  ┆ 0.0078   │
 │ 103749.96 ┆ 0.1425   │
 └───────────┴──────────┘)

In [6]:
gem_bids, gem_asks = pl.DataFrame(data['gem']['bids']), pl.DataFrame(data['gem']['asks'])
_, _ = gem_bids.drop_in_place('timestamp'), gem_asks.drop_in_place('timestamp')
gem_bids, gem_asks = gem_bids.cast(pl.Float64), gem_asks.cast(pl.Float64)
gem_bids, gem_asks = gem_bids.rename({"amount": "qty"}), gem_asks.rename({"amount": "qty"})

In [7]:
gem_bids.head(3), gem_asks.head(3)

(shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 103730.31 ┆ 0.025307 │
 │ 103725.33 ┆ 0.101229 │
 │ 103724.45 ┆ 0.00192  │
 └───────────┴──────────┘,
 shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 103747.74 ┆ 0.025193 │
 │ 103750.02 ┆ 0.025307 │
 │ 103752.93 ┆ 0.00192  │
 └───────────┴──────────┘)

### Merge

In [8]:
all_bids = pl.concat((cb_bids, gem_bids)).sort('price', descending=True)
all_asks = pl.concat((cb_asks, gem_asks)).sort('price', descending=False)
all_bids.head(3), all_asks.head(3)

(shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 103744.42 ┆ 0.078087 │
 │ 103743.48 ┆ 0.002883 │
 │ 103743.36 ┆ 0.000011 │
 └───────────┴──────────┘,
 shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 103744.43 ┆ 0.067058 │
 │ 103747.74 ┆ 0.025193 │
 │ 103748.0  ┆ 0.0078   │
 └───────────┴──────────┘)

In [10]:
all_bids = all_bids.with_columns((pl.col('price') * pl.col('qty')).alias('exec_price'))
all_bids = all_bids.with_columns(pl.col('qty').cum_sum().alias('qty_cumsum'),   pl.col("exec_price").cum_sum().alias("cumsum"))
all_asks = all_asks.with_columns((pl.col('price') * pl.col('qty')).alias('exec_price'))
all_asks = all_asks.with_columns(pl.col('qty').cum_sum().alias('qty_cumsum'),  pl.col("exec_price").cum_sum().alias("cumsum"))
all_bids.head(3), all_asks.head(3)

(shape: (3, 5)
 ┌───────────┬──────────┬─────────────┬────────────┬─────────────┐
 │ price     ┆ qty      ┆ exec_price  ┆ qty_cumsum ┆ cumsum      │
 │ ---       ┆ ---      ┆ ---         ┆ ---        ┆ ---         │
 │ f64       ┆ f64      ┆ f64         ┆ f64        ┆ f64         │
 ╞═══════════╪══════════╪═════════════╪════════════╪═════════════╡
 │ 103744.42 ┆ 0.078087 ┆ 8101.140322 ┆ 0.078087   ┆ 8101.140322 │
 │ 103743.48 ┆ 0.002883 ┆ 299.085191  ┆ 0.08097    ┆ 8400.225513 │
 │ 103743.36 ┆ 0.000011 ┆ 1.141177    ┆ 0.080981   ┆ 8401.36669  │
 └───────────┴──────────┴─────────────┴────────────┴─────────────┘,
 shape: (3, 5)
 ┌───────────┬──────────┬─────────────┬────────────┬─────────────┐
 │ price     ┆ qty      ┆ exec_price  ┆ qty_cumsum ┆ cumsum      │
 │ ---       ┆ ---      ┆ ---         ┆ ---        ┆ ---         │
 │ f64       ┆ f64      ┆ f64         ┆ f64        ┆ f64         │
 ╞═══════════╪══════════╪═════════════╪════════════╪═════════════╡
 │ 103744.43 ┆ 0.067058 ┆ 6956.

In [11]:
target_qty = 1
index_pos = all_bids.select(pl.col('qty_cumsum').search_sorted(target_qty)).item()
index_pos, all_bids.slice(index_pos-1, 3)

(8,
 shape: (3, 5)
 ┌───────────┬──────────┬──────────────┬────────────┬──────────────┐
 │ price     ┆ qty      ┆ exec_price   ┆ qty_cumsum ┆ cumsum       │
 │ ---       ┆ ---      ┆ ---          ┆ ---        ┆ ---          │
 │ f64       ┆ f64      ┆ f64          ┆ f64        ┆ f64          │
 ╞═══════════╪══════════╪══════════════╪════════════╪══════════════╡
 │ 103731.61 ┆ 0.1425   ┆ 14781.754425 ┆ 0.369024   ┆ 38280.57064  │
 │ 103731.0  ┆ 1.0      ┆ 103731.0     ┆ 1.369024   ┆ 142011.57064 │
 │ 103730.31 ┆ 0.025307 ┆ 2625.132     ┆ 1.394331   ┆ 144636.70264 │
 └───────────┴──────────┴──────────────┴────────────┴──────────────┘)

In [12]:
index_pos = all_asks.select(pl.col('qty_cumsum').search_sorted(target_qty)).item()
index_pos, all_asks.slice(index_pos-1, 3)

(21,
 shape: (3, 5)
 ┌───────────┬──────────┬──────────────┬────────────┬───────────────┐
 │ price     ┆ qty      ┆ exec_price   ┆ qty_cumsum ┆ cumsum        │
 │ ---       ┆ ---      ┆ ---          ┆ ---        ┆ ---           │
 │ f64       ┆ f64      ┆ f64          ┆ f64        ┆ f64           │
 ╞═══════════╪══════════╪══════════════╪════════════╪═══════════════╡
 │ 103758.81 ┆ 0.000089 ┆ 9.258399     ┆ 0.598701   ┆ 62116.581883  │
 │ 103759.31 ┆ 0.839956 ┆ 87153.305832 ┆ 1.438658   ┆ 149269.887715 │
 │ 103759.32 ┆ 1.0      ┆ 103759.32    ┆ 2.438658   ┆ 253029.207715 │
 └───────────┴──────────┴──────────────┴────────────┴───────────────┘)

price,qty,exec_price,qty_cumsum,cumsum
f64,f64,f64,f64,f64
103744.43,0.067058,6956.882575,0.067058,6956.882575
103747.74,0.025193,2613.687764,0.092251,9570.57034
103748.0,0.0078,809.2344,0.100051,10379.80474
103749.96,0.1425,14784.3693,0.242551,25164.17404
103750.0,0.0078,809.25,0.250351,25973.42404
…,…,…,…,…
3.6908078e7,7.0000e-8,2.583565,4868.431836,9.3354e8
9.999e7,0.000163,16323.3675,4868.431999,9.3355e8
1.1002e8,4.0000e-8,4.400957,4868.431999,9.3355e8
